In [1]:
import openai
import requests
import copyleaks
import numpy as np
import os
import json
import plotly.express as px
import pandas as pd
pd.set_option('display.max_colwidth', 400)

from DetectGPT.model import GPT2PPLV2 as GPT2PPL
from GLTR.api import LM as GLTR
from transformers import OpenAIGPTLMHeadModel, OpenAIGPTTokenizer, pipeline
from transformers import logging
logging.set_verbosity_error()

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Found API <class 'GLTR.api.LM'> with name gpt-2-small


In [2]:
# WRITER_API = "SibpGkMqgiQcEMsejChg9oetRTCi-rMj9_ndE4SXzg9TdYkh8Hrsba-MYLg6RdSnjAxKBdslLlztEh5E4BkFPyetcS0ce8fboM45zfFIwzE4ZEdoRV91DTCgncPeRwON"
# WRITER_ORG = "556312"
# WRITER_TEAM = "563065"

# COPYLEAKS_API = "b414d289-436f-4906-a461-5c095d69f51e"
# EMAIL_ADDRESS = 'daireiscool@gmail.com'

# SAPLING_API = "B272V7SY9UXAVA6NB8TP6AGBXZBLLZO9"

# 1 indicated AI produced, 0 indicates human produced

In [2]:
def readJSON(location):
    with open(location, encoding = "utf-8") as outfile:
        data = json.load(outfile)
    return pd.DataFrame(data)

In [3]:
def getRobertaScore(text):
    """
    https://huggingface.co/roberta-base-openai-detector
    """
    model = pipeline("text-classification", model="roberta-base-openai-detector")
    vals = model(text, top_k=3)
    flag = sorted(vals, key = lambda x: x["score"])[-1]["label"]
    flag = 1 if flag == "Fake" else 0
    
    return {**{f"roberta_score_{j['label']}": j["score"] for j in vals}, **{"roberta_Label": flag}}
        

In [4]:
def getRobertaLargeScore(text):
    """
    https://huggingface.co/roberta-large-openai-detector
    """
    model = pipeline("text-classification", model="roberta-large-openai-detector")
    vals = model(text, top_k=3)
    flag = sorted(vals, key = lambda x: x["score"])[-1]["label"]
    flag = 1 if flag == "LABEL_1" else 0
    
    return {**{f"roberta_large_score_{j['label']}": j["score"] for j in vals}, **{"roberta_large_Label": flag}}


In [14]:
def getDetectGPTScore(text):
    """
    https://www.arxiv-vanity.com/papers/2301.11305/
    Slow though...
    """
    model = GPT2PPL()
    vals = model(text, len(text), "v1.1")
    flag = 1 - vals[0]
    
    return {"gpt_detect_mean_score": vals[2], 
            "gpt_detect_mean_probability": vals[3], 
            "gpt_detect_Label": flag}

In [7]:
def getGPTZeroScore(text):
    """
    https://gptzero.me/
    """
    model = GPT2PPL()
    vals = model(text, None, "v1")
    flag = 1 - vals[0]["label"]
    
    return {"gpt_zero_Perplexity": vals[0]["Perplexity"], 
            "gpt_zero_Burtiness": vals[0]["Burstiness"], 
            "gpt_zero_Label": flag}

In [8]:
def getGLTRScore(text, threshold = 0.7):
    """
    http://gltr.io/
    """
    gltr = GLTR()
    def f(x):
        return int(np.where(
            x<10, 0, np.where(x<100, 1, np.where(x<1000, 2, 3))))

    def p(x, vals):
        return sum(np.array(vals) == x)/len(vals)

    valsALL = gltr.check_probabilities(text)
    vals = [f(i[0]) for i in valsALL["real_topk"]]
    
    flag = 1 if p(0, vals) > threshold else 0 
    
    return {"gltr_0": p(0, vals), 
            "gltr_1": p(1, vals),
            "gltr_2": p(2, vals),
            "gltr_3": p(3, vals),
            "gltr_Label": flag}

In [ ]:
%%time
roberta_dict = getRobertaScore(textSub)
roberta_large_dict = getRobertaLargeScore(textSub)
gpt_zero_dict = getGPTZeroScore(textSub)
gpt_detect_dict = getDetectGPTScore(textSub)
gltr_dict = getGLTRScore(textSub)

{**roberta_dict, **gpt_zero_dict, **gpt_detect_dict, **roberta_large_dict, **gltr_dict}

In [9]:
def isAIGenerated(text, chunksize = 300, step_size = 300):
    """
    Detect if a function is AI generated or not.
    This will return several scores, which are
        * GPTZero - https://gptzero.me/
        * Roberta - https://huggingface.co/roberta-base-openai-detector
        * Writer - https://writer.com/ai-content-detector/
        * Copyleaks - https://copyleaks.com/ai-content-detector
        * Sapling - https://sapling.ai/programming-language/ai-content-detector/python
        * Potentially GLTR in future... - http://gltr.io/dist/index.html
    
    ::param text: (str)
    ::param chunksize: (int)
    ::param step_size: (int)
    
    ::return: (dict[str: float])
    """
    try:
        data = pd.DataFrame()
        savedText = []
        textSplit = text.split()

        for i in range(len(textSplit)//step_size):
            textSub = " ".join(textSplit[step_size*i:(step_size*i) + chunksize])

            roberta_dict = getRobertaScore(textSub)
            roberta_large_dict = getRobertaLargeScore(textSub)
            gpt_zero_dict = getGPTZeroScore(textSub)
            gpt_detect_dict = getDetectGPTScore(textSub)
            gltr_dict = getGLTRScore(textSub)

            data = data.append({**roberta_dict, **gpt_zero_dict, **gpt_detect_dict, **roberta_large_dict, **gltr_dict}, ignore_index=True)
            savedText = savedText.append(textSub)
        
        data["Text"] = savedText
        return data
    except Exception as e:
        print(e)
        return pd.DataFrame()

In [10]:
allData = pd.DataFrame()
metaData = pd.DataFrame()

for LOC in os.listdir("TranscriptsHistory"):
    print("Starting ", LOC)
    data = readJSON("TranscriptsHistory/"+LOC)
    
    for row in tqdm(data.values):
        symbol = row[0]
        year = row[2]
        quarter = row[3]
        conversation = row[-1]
        
        values = isAIGenerated(conversation)
        metrics = data.describe().reset_index()
        
        metrics["symbol"] = symbol
        metrics["year"] = year
        metrics["quarter"] = quarter
        
        allData = pd.concat([allData, metrics])
        metaData = pd.concat([metaData, values])
    
    allData.to_csv("MetricsOfEarningCallsV1.csv")
    allData.to_excel("MetaOfEarningCallsV1.xlsx")

Starting  AAPL.json


  0%|          | 0/44 [00:00<?, ?it/s]

Cannot describe a DataFrame without columns
Starting  AON.json


  0%|          | 0/44 [00:00<?, ?it/s]

Starting  C.json


...

  0%|          | 0/44 [00:00<?, ?it/s]

Starting  UNH.json


  0%|          | 0/44 [00:00<?, ?it/s]

Starting  XOM.json


  0%|          | 0/44 [00:00<?, ?it/s]